In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
#import analyze_covid
#import airport_data_explore

# Unload stored dataframes from other notebooks within directory
%store -r

In [4]:
# Haversine Caluclation (Distance between two lat and lon coordinates) retrieved from:
# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula/21623206

def haversine(lat1, lon1, lat2, lon2):
    p = np.pi / 180
    a = 0.5 - np.cos((lat2 - lat1) * p) / 2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p )) / 2
    n = 12742 * np.arcsin(np.sqrt(a))
    return n  

In [5]:
def getStates():
    return usLocations['Province/State'].unique()

In [6]:
def closest(state):
    d = []
    
    t = usLocations[usLocations["Province/State"] == state]
    i = airportLocationsUS[airportLocationsUS["State"] == state]
    size = len(t)
    size2 = len(i)

    for a in range(size2):
        for s in range(size):
            airport = {
                    'country': t.iloc[s]["Country/Region"],
                    'state/province': t.iloc[s]["Province/State"],
                    'county': t.iloc[s]["County"],
                    'lat': t.iloc[s]["Latitude"],
                    'long': t.iloc[s]["Longitude"],
                    'airport': i.iloc[a]["AirportName"],
                    'distance(km)': haversine(t.iloc[s]["Latitude"], t.iloc[s]["Longitude"], i.iloc[a]["lat"], i.iloc[a]["long"])
                }
            d.append(airport)
    d = sorted(d, key = lambda x: x['distance(km)'])
    return d

In [7]:
def calcNearestAirport(state):
    d = []
    for s in state:
        d.append(closest(state))
    return d[0]

#calcNearestAirport('California')        

In [8]:
statesList = getStates()
listDf = []

for s in statesList:
    i = calcNearestAirport(s)
    listDf.append(pd.DataFrame.from_dict(i))
    
main = pd.concat(listDf)

In [9]:
listDf2 = []

for s in statesList:
    df = main[main["state/province"] == s].sort_values(by = ['county', 'distance(km)']).reset_index(drop = True)
    df = df.iloc[::2]
    listDf2.append(df)

usa = pd.concat(listDf2).reset_index(drop = True)
usa

,airport,country,county,distance(km),lat,long,state/province
0,LaGuardia,US,Albany,203.305377,42.600603,-73.977239,New York
1,LaGuardia,US,Allegany,383.150746,42.257484,-78.027505,New York
2,LaGuardia,US,Bronx,8.740585,40.852093,-73.862828,New York
3,LaGuardia,US,Broome,223.265360,42.159032,-75.813261,New York
4,LaGuardia,US,Cattaraugus,432.353848,42.247782,-78.679231,New York
5,LaGuardia,US,Cayuga,325.527992,42.912617,-76.557316,New York
6,LaGuardia,US,Chautauqua,485.116600,42.227692,-79.366918,New York
7,LaGuardia,US,Chemung,284.699688,42.138911,-76.763880,New York
8,LaGuardia,US,Chenango,239.575861,42.494300,-75.608876,New York
9,LaGuardia,US,Clinton,441.887337,44.745309,-73.678754,New York


In [11]:
jfk = usa[usa["airport"] == "John F. Kennedy International"].reset_index(drop = True)
jfk

,airport,country,county,distance(km),lat,long,state/province
0,John F. Kennedy International,US,Kings,14.461959,40.636182,-73.949356,New York
1,John F. Kennedy International,US,Lewis,375.091055,43.784416,-75.449040,New York
2,John F. Kennedy International,US,Nassau,19.097665,40.740665,-73.589419,New York
3,John F. Kennedy International,US,Queens,7.905192,40.710881,-73.816847,New York
4,John F. Kennedy International,US,Suffolk,86.422753,40.883201,-72.801217,New York


In [13]:
lga = usa[usa["airport"] == "LaGuardia"].reset_index(drop = True)
lga 

,airport,country,county,distance(km),lat,long,state/province
0,LaGuardia,US,Albany,203.305377,42.600603,-73.977239,New York
1,LaGuardia,US,Allegany,383.150746,42.257484,-78.027505,New York
2,LaGuardia,US,Bronx,8.740585,40.852093,-73.862828,New York
3,LaGuardia,US,Broome,223.265360,42.159032,-75.813261,New York
4,LaGuardia,US,Cattaraugus,432.353848,42.247782,-78.679231,New York
5,LaGuardia,US,Cayuga,325.527992,42.912617,-76.557316,New York
6,LaGuardia,US,Chautauqua,485.116600,42.227692,-79.366918,New York
7,LaGuardia,US,Chemung,284.699688,42.138911,-76.763880,New York
8,LaGuardia,US,Chenango,239.575861,42.494300,-75.608876,New York
9,LaGuardia,US,Clinton,441.887337,44.745309,-73.678754,New York


In [15]:
sea = usa[usa["state/province"] == "Washington"].reset_index(drop = True)
sea

,airport,country,county,distance(km),lat,long,state/province
0,Seattle-Tacoma International,US,Adams,287.814681,46.982998,-118.560173,Washington
1,Seattle-Tacoma International,US,Asotin,412.998825,46.188944,-117.202285,Washington
2,Seattle-Tacoma International,US,Benton,251.717423,46.239470,-119.512083,Washington
3,Seattle-Tacoma International,US,Clallam,137.717128,48.047546,-123.922632,Washington
4,Seattle-Tacoma International,US,Clark,186.715529,45.775680,-122.482920,Washington
5,Seattle-Tacoma International,US,Columbia,358.505696,46.294429,-117.905198,Washington
6,Seattle-Tacoma International,US,Douglas,198.692532,47.736134,-119.692937,Washington
7,Seattle-Tacoma International,US,Ferry,304.349266,48.471143,-118.515404,Washington
8,Seattle-Tacoma International,US,Garfield,378.867203,46.431975,-117.545403,Washington
9,Seattle-Tacoma International,US,Garfield County,378.144513,46.451975,-117.545403,Washington


In [17]:
atl = usa[usa["state/province"] == "Georgia"].reset_index(drop = True)
atl

,airport,country,county,distance(km),lat,long,state/province
0,Hartsfield-Jackson Atlanta International,US,Appling,290.408424,31.748472,-82.289091,Georgia
1,Hartsfield-Jackson Atlanta International,US,Bacon,296.355645,31.554565,-82.459365,Georgia
2,Hartsfield-Jackson Atlanta International,US,Baldwin,126.801083,33.068823,-83.247017,Georgia
3,Hartsfield-Jackson Atlanta International,US,Barrow,76.574848,33.991487,-83.714363,Georgia
4,Hartsfield-Jackson Atlanta International,US,Ben Hill,237.586339,31.760959,-83.221454,Georgia
5,Hartsfield-Jackson Atlanta International,US,Bibb,114.378628,32.809042,-83.704892,Georgia
6,Hartsfield-Jackson Atlanta International,US,Brantley,356.118400,31.195766,-81.977440,Georgia
7,Hartsfield-Jackson Atlanta International,US,Bryan,332.913766,32.009793,-81.440029,Georgia
8,Hartsfield-Jackson Atlanta International,US,Burke,234.670368,33.059180,-81.999287,Georgia
9,Hartsfield-Jackson Atlanta International,US,Calhoun,235.294264,31.531698,-84.627251,Georgia


In [19]:
lax = usa[usa["airport"] == "Los Angeles International"].reset_index(drop = True)
lax

,airport,country,county,distance(km),lat,long,state/province
0,Los Angeles International,US,Alpine,533.003970,38.596786,-119.822359,California
1,Los Angeles International,US,Amador,540.051664,38.445831,-120.656960,California
2,Los Angeles International,US,Calaveras,511.882961,38.205371,-120.552913,California
3,Los Angeles International,US,Del Norte,991.944032,41.742283,-123.897406,California
4,Los Angeles International,US,Imperial,299.041914,33.039308,-115.366904,California
5,Los Angeles International,US,Inyo,299.657033,36.511121,-117.411198,California
6,Los Angeles International,US,Kern,158.658748,35.343294,-118.727780,California
7,Los Angeles International,US,Kings,269.720013,36.074091,-119.815997,California
8,Los Angeles International,US,Lake,692.748906,39.101243,-122.753624,California
9,Los Angeles International,US,Los Angeles,43.923416,34.308284,-118.228241,California


In [22]:
jfk.to_csv('jfk.csv')  
lga.to_csv('lga.csv') 
sea.to_csv('sea.csv')
atl.to_csv('atl.csv')
lax.to_csv('lax.csv')